In [1]:
Experiments_file='/home/kate/Research/YearBuilt/Experiments/DwellingExperiments.xlsx'
AllExperiments_tab='Experiments'
Experiment_name='PartDepFWC'
#Experiment configuration:  features we want to produce PartialDependency
#each line in the file contains a feature name and a type:Categorical or Continuous
#Categorical features values will be process all as is. Continuous only sorted(np.linspace(np.percentile(dataset_temp[f], 0.1),np.percentile(dataset_temp[f], 99.5),50))
Experiment_Features_tab='%s Features'%Experiment_name
#The full set of features the tested model is based on
#each line in the file contains the model name and set of features to built a dataset for SageMaker
#only one,first row is taken into account
Experiment_tab='%s Models'%Experiment_name

#Looks like Trial name should be unique in my environment, not in the experiment it belongs to
Trial_name='%s-PreparingData'%Experiment_name
 
bucket='kdproperty'
path_to_data_file='/Data/'


instance_type='ml.t3.2xlarge'
instance_count=1

In [2]:
import boto3
import sys
import time
import pandas as pd
import numpy as np

import sagemaker
from sagemaker import get_execution_role
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput

from sagemaker.analytics import ExperimentAnalytics

In [3]:
sys.path.append('/home/kate/Research/YearBuilt/Notebooks/Property')
import ExperimentsUtils as eu

In [4]:
#delete experiment and trials but not output files associated with jobs in experiments
eu.cleanup_experiment(Experiment_name)

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


PartDepFWC is a new experiment. nothing to delete


In [5]:
eu.create_experiment(Experiment_name)
eu.create_trial(Experiment_name,Trial_name)

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


In [6]:
experiments = pd.read_excel(open(Experiments_file, 'rb'), sheet_name=AllExperiments_tab)
target=experiments[experiments['Experiment']==Experiment_name]['Target'].values[0]
print(target)
data_file=experiments[experiments['Experiment']==Experiment_name]['Dataset'].values[0]
print(data_file)

hasclaim_water
dwelling_basedata_v4.csv


In [7]:
Features = pd.read_excel(open(Experiments_file, 'rb'), sheet_name=Experiment_Features_tab)
Features

,Feature,Type
0,cal_year - yearbuilt,Continuous


In [8]:
#take into account only one,first row 
models = pd.read_excel(open(Experiments_file, 'rb'), sheet_name=Experiment_tab)
for index, row in models.iterrows():
    model_complete_featureset=row[1:51].tolist()
    model_complete_featureset=[x for x in model_complete_featureset if str(x) != 'nan']
    break
model_complete_featureset

['usagetype_encd',
 'cova_deductible',
 'cova_limit ',
 'sqft',
 'cal_year - yearbuilt',
 'landlordind',
 'water_risk_3_blk',
 'constructioncd_encd']

In [9]:
region = boto3.session.Session().region_name
role = 'arn:aws:iam::757107622481:role/service-role/AmazonSageMaker-ExecutionRole-20200819T131882'
sagemaker_session = sagemaker.session.Session(default_bucket=bucket)

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


In [10]:
%%writefile preprocessingDataForPartialDependency.py

#The code creates a separate dataset for each feature with all possible combination of feature values and the rest of the data
#dataset for SageMaker are the same structure: no headers, the first column is a target and the rest are features


import argparse
import os
import pandas as pd
import numpy as np


if __name__=='__main__':
    
    parser = argparse.ArgumentParser()
    parser.add_argument('--data_file', type=str)
    parser.add_argument('--model_complete_featureset', type=str)      
    parser.add_argument('--featureset', type=str)    
    parser.add_argument('--featuretypes', type=str)     
    args, _ = parser.parse_known_args()    
    print('Received arguments {}'.format(args))
    
    model_complete_featureset=args.model_complete_featureset.split(',')
    featureset=args.featureset.split(',')
    featuretypes=args.featuretypes.split(',')
    input_data_path = os.path.join('/opt/ml/processing/input', args.data_file)

    
    print('Reading input data from {}'.format(input_data_path))
    dataset = pd.read_csv(input_data_path, error_bad_lines=False, index_col=False)
    
     
   
    for feature,ftype in zip(featureset,featuretypes):
        print(feature,ftype)
        dataset_feature = pd.DataFrame()    
        #dataset_temp = dataset[model_complete_featureset].copy()  
        dataset_temp = pd.DataFrame()
        for f in model_complete_featureset:
            dataset_temp[f]=dataset.eval(f)
        if ftype=='Continuous':
            # continuous
            grid = sorted(np.linspace(np.percentile(dataset_temp[feature], 0.1),
                       np.percentile(dataset_temp[feature], 99.5),
                          50))
        else:
            #categorical
            grid = sorted(dataset_temp[feature].unique())        
 
        for i, val in enumerate(grid):
            dataset_temp[feature] = val
            dataset_feature=dataset_feature.append(dataset_temp)
        #save in parts if large dataset
        if ftype=='Continuous':
            parts = np.array_split(dataset_feature, 5)
            
            for i,p in enumerate(parts):
                output_data_path = os.path.join('/opt/ml/processing/output', '%s_%s.csv'%(feature,i))
                p.to_csv(output_data_path,header=False,index=False)
        else:   
            output_data_path = os.path.join('/opt/ml/processing/output', '%s.csv'%feature)
            dataset_feature.to_csv(output_data_path,header=False,index=False)
        

Overwriting preprocessingDataForPartialDependency.py


In [11]:
processors=list()

featureset=Features['Feature'].tolist()
featuretypes=Features['Type'].tolist()

data_processor = SKLearnProcessor(framework_version='0.20.0',
                                     role=role,
                                     instance_type=instance_type,
                                     instance_count=instance_count)

data_processor.run(code='preprocessingDataForPartialDependency.py',
                        inputs=[ProcessingInput(
                        source='s3://%s%s'%(bucket,path_to_data_file+data_file),
                        destination='/opt/ml/processing/input')],
                        outputs=[ProcessingOutput(output_name='output', source='/opt/ml/processing/output', destination='s3://%s%sPartialDependencyInputData/%s'%(bucket,path_to_data_file,Experiment_name)),                                                          
                                ],
                        arguments=['--data_file',data_file,
                                 '--model_complete_featureset',','.join(model_complete_featureset).replace(' ',''),
                                 '--featureset', ','.join(featureset).replace(' ',''),
                                 '--featuretypes', ','.join(featuretypes).replace(' ','')],
                        experiment_config = {
        'ExperimentName': Experiment_name ,
        'TrialName' : Trial_name,
        'TrialComponentDisplayName' : '%s'%(Trial_name,),},
                        wait=False
                        )
processors.append(data_processor)

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:sagemaker.image_uris:Same images used for training and inference. Defaulting to image scope: inference.
INFO:sagemaker.image_uris:Defaulting to only available Python version: py3
INFO:sagemaker:Creating processing-job with name sagemaker-scikit-learn-2020-12-31-20-28-01-844



Job Name:  sagemaker-scikit-learn-2020-12-31-20-28-01-844
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://kdproperty/Data/dwelling_basedata_v4.csv', 'LocalPath': '/opt/ml/processing/input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-west-2-757107622481/sagemaker-scikit-learn-2020-12-31-20-28-01-844/input/code/preprocessingDataForPartialDependency.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'output', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://kdproperty/Data/PartialDependencyInputData/PartDepFWC', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}}]


In [12]:
#wait till the rest of processing jobs complete
eu.wait_processing_jobs(processors=processors,check_every_sec=10,print_every_n_output=10,wait_min=60)

Processing job sagemaker-scikit-learn-2020-12-31-20-28-01-844 status: InProgress
Continue waiting...
Processing job sagemaker-scikit-learn-2020-12-31-20-28-01-844 status: InProgress
Continue waiting...
Processing job sagemaker-scikit-learn-2020-12-31-20-28-01-844 status: InProgress
Continue waiting...
Processing job sagemaker-scikit-learn-2020-12-31-20-28-01-844 status: InProgress
Continue waiting...
Processing job sagemaker-scikit-learn-2020-12-31-20-28-01-844 status: InProgress
Continue waiting...
Processing job sagemaker-scikit-learn-2020-12-31-20-28-01-844 status: InProgress
Continue waiting...
All Processing Jobs are Completed


In [13]:
trial_component_analytics = ExperimentAnalytics(
    experiment_name=Experiment_name
)
trial_comp_ds = trial_component_analytics.dataframe()
trial_ds=trial_comp_ds[trial_comp_ds['DisplayName'].str.contains(Trial_name)].copy()
trial_ds=trial_ds[['DisplayName','output - Value']]
trial_ds.columns=['Trial Component','Data']
trial_ds

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


,Trial Component,Data
0,PartDepFWC-PreparingData,s3://kdproperty/Data/PartialDependencyInputDat...


In [15]:
#Saving into the Experiment log location of the created  dataset
eu.SaveToExperimentLog(Experiments_file, '%s InputData'%Experiment_name, trial_ds)